# Merge all data sets

This notebook accumulates each data set and merges them on date.

### Import relevant libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import string
from datetime import datetime
import re
import pickle
from functools import reduce
from collections import Counter

%matplotlib inline

#### Set pandas options

In [ ]:
pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)
pd.set_option('precision', 3)

### Import MarketWatch articles

In [ ]:
pickles = ['links', 'econ', 'fed', 'strong', 'soft']

for p in pickles:
    with open('/Users/samfunk/ds/metis/project_luther/' + p + '.pkl', 'rb') as f:
        globals()[str(p)] = pickle.load(f)

### Convert article info into dataframe and format date and url columns

In [ ]:
def to_df(dictfile, colname):
    '''
    Make dataframes out of raw article dictionaries. Format dates, urls, and limit to relevant time horizon
    ---
    IN: dictfile = imported article dictionaries, colname = dataframe column name for merging
    OUT: formatted dataframe
    '''
    
    df = pd.DataFrame()
    
    try:
        dictlist = [x for x in dictfile.items()]
    except:
        dictlist = dictfile

    df['article'] = [x[0].lower() for x in dictlist]
    df['datetime'] = [x[1]['date'] for x in dictlist]
    df['url'] = [x[1]['url'] for x in dictlist]
    if colname != 'links':
        df[colname] = 1
    
    regex = re.compile('[%s]' % re.escape(string.punctuation.replace(':', '')))
    df['datetime'] = df['datetime'].apply(lambda x: regex.sub('', x))

    df['time'] = df['datetime'].apply(lambda x: re.search(r'^(.+?\s.+?)\s', x)[1])
    df['time_string'] = df['time'].apply(lambda x: datetime.strptime(x, '%I:%M %p').time())

    df['date'] = df['datetime'].apply(lambda x: re.search(r'([A-Z].*)', x)[1])
    df['date'] = df['date'].apply(lambda x: re.sub(r'([A-Za-z]{3})[a-z]*', r'\1', x))
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%b %d %Y'))
    df['date_string'] = df['date'].apply(lambda x: x.date())

    df['datetime'] = df.apply(lambda x: datetime.combine(x['date_string'], x['time_string']), axis=1)

    df['month'] = df['datetime'].apply(lambda x: x.month)
    df['day'] = df['datetime'].apply(lambda x: x.weekday())

    df['url'] = df['url'].apply(lambda x: 'http://marketwatch.com' + x if 'http' not in x else x)
    
    df = df.sort_values('datetime')
    
    df = df[(df['month'] > 5) & (df['month'] < 10)]
    df = df[df['day'] < 5]

    return df.copy()
    

### Merge (concatenate) all dataframes into `master`

In [ ]:
df_links = to_df(links, 'links')
df_econ = to_df(econ, 'econ')
df_fed = to_df(fed, 'fed')
df_strong = to_df(strong, 'strong')
df_soft = to_df(soft, 'soft')
data = [df_links, df_econ, df_fed, df_strong, df_soft]

master = pd.concat(data, ignore_index=True)
master.shape

### Format `master` and add additional columns of variable transformations

In [ ]:
c = Counter(master.datetime)
dups = [n for n in c if c[n] > 1]
mask = master.datetime.isin(dups)
result = master.loc[(~mask) | ((mask) & (master.econ > 0)) | ((mask) & (master.fed > 0)) | ((mask) & (master.strong > 0)) | ((mask) & (master.soft > 0)), :]

master = result.drop_duplicates()
master = master.fillna(0)
master = master[master['date'] != '2016-07-08']

count = master.groupby('date')['article'].count().reset_index().set_index('date')
sums = master.groupby('date')['econ', 'fed', 'strong', 'soft'].sum().reset_index().set_index('date')
master = sums.join(count, how='left')

master['econ_day'] = master['econ'] / master['article']
master['strong_day'] = master['strong'] / master['article']
master['strong_soft'] = master['strong'] + master['soft']
master['ss_day'] = master['strong_soft'] / master['article']

master['strong_econ'] = master['strong'] / master['econ']
master['ss_econ'] = master['strong_soft'] / master['econ']

master.shape

## Read in and merge Fed calendar data

In [ ]:
with open('/Users/samfunk/ds/metis/project_luther/calendar.pkl', 'rb') as f:
    calendar = pickle.load(f)

calendar = calendar.groupby(calendar.index).first().reset_index().set_index('date')

master = master.join(calendar, how='left')
master = master.fillna(0)

In [ ]:
master.shape

## Read in and merge Google Trends data

In [ ]:
with open('/Users/samfunk/ds/metis/project_luther/trends.pkl', 'rb') as f:
    trends = pickle.load(f)
    
trends.rename(columns={'lag1': 'trend_lag1', 'lag2': 'trend_lag2', 'federal reserve': 'federal_reserve', 'unemployment rate': 'unemployment'}, inplace=True)

master = master.join(trends, how='left')

In [ ]:
master.shape

## Read in and merge SPX data

In [ ]:
spx = pd.read_csv('/Users/samfunk/Downloads/GSPC.csv')

In [ ]:
spx['Date'] = spx['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
spx = spx.sort_values(by='Date').set_index('Date')

spx = spx['06-01-2017':'10-02-2017']

spx['lag1'] = spx['Close'].diff()
spx['perc_change'] = spx['Close'].diff() / spx['Close'].shift()

spx = spx[['Close', 'lag1', 'perc_change']]

master = master.join(spx, how='left')
master = master[np.isfinite(master.lag1)]

master.rename(columns={'lag1': 'spx_lag', 'Close': 'close'}, inplace=True)
master['abs_lag'] = master['spx_lag'].apply(lambda x: abs(x))

In [ ]:
master.shape

## Save `master` data frame

In [ ]:
with open('/Users/samfunk/ds/metis/project_luther/master.pkl', 'wb') as f:
    pickle.dump(master, f)